In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from collections import defaultdict
url = 'https://www.billboard.com/charts/hot-100/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
%pip install spotipy
CLIENT_ID = "4109b2d9f5014671863bb2df1d1fbdd3"
CLIENT_SECRET = "5cb719fae80c4191a9c0b288f51bfe50"
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET))

In [ ]:
def get_artist_name(artist_id):
    artist_name = sp.artist(artist_id)
    return artist_name['name']

In [ ]:
def get_songs(soup):
    songs = soup.find_all("h3", attrs={'class': 'c-title'})
    i = 0
    x = 0
    songs_list_100 = []
    for song in songs:
        i += 1
        if i > 6 and i - x * 4 == 7 and i < 407:
            x += 1
            song = song.get_text().strip()
            songs_list_100.append(song)
    #print(songs_list_100)
    return songs_list_100

In [ ]:
def get_bands(soup):                                                        # Wood have worked better with ReGex?
    bands = soup.find_all("span", attrs={'class': 'c-label'})
    #print(bands)
    filtered_bands = []
    for band in bands:
        if band.get_text().strip() != "NEW" and band.get_text().strip() != "RE-\nENTRY":
            filtered_bands.append(band)
    #print(filtered_bands)
    #print(len(filtered_bands))
    j = 0
    y = 0
    band_list_100 = []
    for band in filtered_bands:
        j += 1
        if j > 1 and j - y * 8 == 2 and j < 795:
            y += 1
            band = band.get_text().strip()
            band_list_100.append(band)
    #print(band_list_100)
    return band_list_100

In [ ]:
def merge_bands_songs(band_list_100, songs_list_100):
    top_acts = defaultdict(list)
    for k, v in zip(band_list_100, songs_list_100):
        top_acts[k].append(v)
    return top_acts

In [ ]:
def look_up_100(top_acts, artist_name):
    for k,v in top_acts.items():
        if artist_name == k:
            print(f'The artist of your song has these songs in the Billboard 100: {v}\n')

In [ ]:
def get_track_id(your_song):
    results = sp.search(q=your_song, limit=1)
    track_id = results['tracks']['items'][0]['id']
    return track_id

In [ ]:
def get_artist_id(track_id):
    results2 = sp.track(track_id, market=None)
    artist_id = results2['album']['artists'][0]['id']
    return artist_id

In [ ]:
def get_top_songs_artist(artist_id):
    results3 = sp.artist_top_tracks(artist_id)
    track_list = []
    for track in results3['tracks']:
        track_list.append(track['name'])
    print(f'\nYou might also like these songs from the same artist:{track_list}\n\n')

In [ ]:
def recommendations():
    
    
    your_song = input("Please enter your favorite song")
    
    
    # Get top songs of the artist from spotify:
    
    track_id = get_track_id(your_song)
    
    artist_id = get_artist_id(track_id)
    
    get_top_songs_artist(artist_id)
    
    
    # Get songs from the artist from Billboard 100:
    
    songs_list_100 = get_songs(soup)

    
    band_list_100 = get_bands(soup)

   
    
    top_acts = merge_bands_songs(band_list_100, songs_list_100)

  

    artist_name = get_artist_name(artist_id) # artist_id already received earliyer

   
    
    look_up_100(top_acts, artist_name)
    

In [ ]:
recommendations()